#### This notebook explains how to:
(1) download raw benchmarking data to your local computer 

(2) read raw benchmarking data from S3

(3) read precomputed volume/skeleton benchmarking data via Neuroglancer

#### Quick notes on the benchmarking data:

In octree format, data is labled in folders 1-50. 1-25 correspond with the 1-25 "test" raw data and 26-50 correspond with the 1-25 "validation" raw data

Known issues with a few of the files: 

- test_10 (#10),test_9 (#9) - didnt seem to have good swc alignment

- test_24 (#24) - issues with the image

- validation_11 (#37) - seems to be a shift between swcs and the image


# Downloading Raw Benchmarking Data with AWS CLI

###### This will download the benchmarking data in .tif and .swc format to a local computer

### 1) Installing AWS CLI

You can install AWS CLI for any major operating system:

#### macOS 
(the full documentation uses pip, but Homebrew works more seamlessly):
Documentation: https://docs.aws.amazon.com/cli/latest/userguide/cli-install-macos.html 

```brew install awscli```
#### Linux 
Documentation: https://docs.aws.amazon.com/cli/latest/userguide/awscli-install-linux.html 

```$ pip install awscli```

#### Windows
Documentation: https://docs.aws.amazon.com/cli/latest/userguide/awscli-install-windows.html
Installer: http://docs.aws.amazon.com/cli/latest/userguide/awscli-install-windows.html

### 2) Configuring AWS CLI

Run ```aws configure``` in any command line and follow the prompts.

You will be asked for 
- AWS Access Key ID
- AWS Secrety Access Key
- Default Region Name: us-east-1, us-east-2, us-west-1, us-west-2, ...
- Default output format: json, table, text

### 3.1) Downloading .tif files

Run the command ```aws s3 sync s3://open-neurodata/brainlit/benchmarking_data/tif-files/ sample-tif-location```

Replace "sample-tif-location" with your respective download destination.

### 3.2) Accessing AND Visualizing .tif files

In [ ]:
from pathlib import Path
import numpy as np
from skimage import io

This loops through and reads the .tif files in the download directory. Alter data_dir to point to your download location.

In [ ]:
%%capture
data_dir = Path().resolve().parents[5] / "Downloads" / "sample-tif-location"
im_files = list(data_dir.glob("**/*.tif"))

for im_num, im_file in enumerate(im_files):
    print(f"Image {im_num}/{len(im_files)}")
    print(im_file)
    f = im_file.parts[-1][:-8] + "-gfp.tif" 
    im = io.imread(im_file, plugin="tifffile") 

The below code can visualize a specified .tif file. To visualize tif files with .swc overlaid, run the data through the image mask generation notebook.

In [ ]:
import napari

file_name = "test_10-gfp.tif" # Can change to any image (test 1-25, validation 1-25)

im_file = data_dir / file_name
im = io.imread(im_file, plugin="tifffile") 
    
with napari.gui_qt():
    viewer = napari.Viewer(ndisplay=3)
    viewer.add_image(im)

### 4.1) Downloading .swc files

Run the command ```aws s3 sync s3://open-neurodata/brainlit/benchmarking_data/Manual-GT/ sample-tif-location/sample-swc-location```

Replace "sample-swc-location" with your respective download destination. Keep it within the .tif folder to maintain octree format

### 4.2) Accessing .swc files

This loops through the .swc files in the download directory. Alter data_dir to point to your file download location.


In [ ]:
%%capture
data_dir = Path().resolve().parents[5] / "Downloads" / "sample-swc-location" 
swc_files = list(data_dir.glob("**/*.swc"))

for swc_num, swc_file in enumerate(swc_files):
    print(f"SWC {swc_num}/{len(swc_files)}")
    print(swc_file)
    image_name = swc_file.parts[6]
    swc_num = swc_file.parts[7]

### 5) Visualizing 

In [ ]:
from mouselight_code.src.benchmarking_params import brain_offsets, vol_offsets, scales, type_to_date
from mouselight_code.src import read_swc
from brainlit.utils.swc import df_to_graph, graph_to_paths
from pathlib import Path
import numpy as np
from skimage import io
from mouselight_code.src.visualize import napari_viewer

In [ ]:
im_dir = Path("C:/Users/shrey/Downloads/sample-tif-location")

gfp_files = list(im_dir.glob("**/*-gfp.tif"))

swc_base_path = im_dir / "sample-swc-location"

In [ ]:
for im_num, im_path in enumerate(gfp_files):
    
    print(f"Image {im_num+1}/{len(gfp_files)}")
    print(im_path)
    
    f = im_path.parts[-1][:-8].split("_")
    image = f[0]
    date = type_to_date[image]
    num = int(f[1])

    scale = scales[date]
    brain_offset = brain_offsets[date]
    vol_offset = vol_offsets[date][num]
    im_offset = np.add(brain_offset, vol_offset)

    lower = int(np.floor((num - 1) / 5) * 5 + 1)
    upper = int(np.floor((num - 1) / 5) * 5 + 5)
    dir1 = date + "_" + image + "_" + str(lower) + "-" + str(upper)
    dir2 = date + "_" + image + "_" + str(num)
    swc_path = swc_base_path / dir1 / dir2

    swc_files = list(swc_path.glob("**/*.swc"))
    im = io.imread(im_path, plugin="tifffile")
    print(f"Image shape: {im.shape}")

    paths_total = []
    for swc_num, swc in enumerate(swc_files):
        if "cube" in swc.parts[-1]:
            # skip the bounding box swc
            continue

        df, swc_offset, _, _, _ = read_swc.read_swc(swc)

        offset_diff = np.subtract(swc_offset, im_offset)
        G = df_to_graph(df)

        paths = graph_to_paths(G)
       
        for path_num, p in enumerate(paths):
            pvox = (p + offset_diff) / (scale) * 1000
            paths_total.append(pvox)
            
        napari_viewer(np.swapaxes(im,0,2), shapes=paths_total)

# Read benchmarking data from S3 with boto3
###### This will load the benchmarking .tif images from S3 and can either display them on napari or a local image viewer

### 1) Install and configure boto3

Pip install boto3 if not previously installed.
Save a credentials file in ```~/.aws/credentials.ini```


File format: 

[default]

aws_access_key_id = xxxxxxxxxxxxxxxx

aws_secret_access_key = xxxxxxxxxxxxxxxxxxxxxxxx


### 2) Read the .tif files

In [ ]:
import boto3
import os
import numpy as np
from PIL import Image
import io
from io import BytesIO
from pathlib import Path
from skimage import io

f_list = ['test_1-gfp.tif', 'test_2-gfp.tif', 'test_3-gfp.tif', 'test_4-gfp.tif', 'test_5-gfp.tif', 'test_6-gfp.tif',
 'test_7-gfp.tif', 'test_8-gfp.tif', 'test_9-gfp.tif', 'test_10-gfp.tif', 'test_11-gfp.tif', 'test_12-gfp.tif',
 'test_13-gfp.tif', 'test_14-gfp.tif', 'test_15-gfp.tif', 'test_16-gfp.tif', 'test_17-gfp.tif', 'test_18-gfp.tif',
 'test_19-gfp.tif', 'test_20-gfp.tif', 'test_21-gfp.tif', 'test_22-gfp.tif', 'test_23-gfp.tif', 'test_24-gfp.tif',
 'test_25-gfp.tif', 'validation_1-gfp.tif', 'validation_2-gfp.tif', 'validation_3-gfp.tif', 'validation_4-gfp.tif',
 'validation_5-gfp.tif', 'validation_6-gfp.tif', 'validation_7-gfp.tif', 'validation_8-gfp.tif', 'validation_9-gfp.tif',
 'validation_10-gfp.tif', 'validation_11-gfp.tif', 'validation_12-gfp.tif', 'validation_13-gfp.tif', 'validation_14-gfp.tif',
 'validation_15-gfp.tif', 'validation_16-gfp.tif', 'validation_17-gfp.tif', 'validation_18-gfp.tif', 'validation_19-gfp.tif',
 'validation_20-gfp.tif', 'validation_21-gfp.tif', 'validation_22-gfp.tif', 'validation_23-gfp.tif', 'validation_24-gfp.tif',
 'validation_25-gfp.tif']

#HOME = Path("C:/Users/shrey") #Change to specify Home directory if aws config file is not being found.
s3 = boto3.resource('s3')
my_bucket = s3.Bucket('open-neurodata')

#### Read and Visualize a specific .tif file

In [ ]:
file = 'test_10-gfp.tif' #Replace with specific .tif file name
print('File name: ', file)

prefix = 'benchmarking_data/tif-files/' + file
objects = my_bucket.objects.filter(Prefix = prefix)
  
for obj in objects:
    path, filename = os.path.split(obj.key)
    im = Image.open(obj.get()['Body'])
    im.show() # Can't visualize on Napari yet (?) but this line will launch image in local image viewer
    

In [ ]:
print('Image to Numpy Array: ') #converting to numpy array
print(np.array(im))

print('File: ', file) #viewing image in notebook
im


In [ ]:
#Can't vizualize in napari YET, "TypeError: 'TiffImageFile' object is not iterable"
import napari
with napari.gui_qt():
    viewer = napari.Viewer(ndisplay=3)
    viewer.add_image(im)
    #viewer.add_labels(labels)

#### Read ALL 50 .tif files & visualize on local image viewer
Files should pop up in local image viewer (For Windows, it's the Photos app)

In [ ]:
%%capture
for file in f_list:
    print('File name: ', file)
    prefix = 'benchmarking_data/tif-files/' + file
    objects = my_bucket.objects.filter(Prefix=prefix)
    
    for obj in objects:
        path, filename = os.path.split(obj.key)
        im = Image.open(obj.get()['Body'])
        im.show()

# Read benchmarking data from S3 with Neuroglancer
###### This will load the benchmarking data from precomputed volume and skeleton form

In [ ]:
from brainlit.utils import session, upload_benchmarking
dest = "s3://open-neurodata/brainlit/benchmarking_data/test"
dest_segments = "s3://open-neurodata/brainlit/benchmarking_data/test"

In [ ]:
sess._get_voxel(1,3)


In [ ]:
%%capture
sess = session.NeuroglancerSession(url=dest, url_segments=dest_segments, mip=0)  # create session object object
#img, bounds, vox = sess.pull_chunk(1, 1, 1)
#img, bbox, vox = sess.pull_voxel(2, v_id = 1, radius = 0)
#img, bounds, vertices = sess.pull_vertex_list(1, list(range(1,187)), 0, expand=False)  # get image containing some data


In [ ]:
G = sess.get_segments(1, bbox = [330,300,100])
paths = graph_to_paths(G)
print(f"Selected volume contains {G.number_of_nodes()} nodes and {len(paths)} paths")

In [ ]:
import napari
with napari.gui_qt():
    viewer = napari.Viewer(ndisplay=3)
    viewer.add_image(img)
    viewer.add_shapes(data=paths, shape_type='path', edge_width=0.1, edge_color='blue', opacity=0.1)
    viewer.add_points(vox, size=1, opacity=0.5)

## Misc, Ignore 

In [ ]:

from mouselight_code.src.benchmarking_params import brain_offsets, vol_offsets, scales, type_to_date
from mouselight_code.src import read_swc
from brainlit.utils.swc import df_to_graph, graph_to_paths
from pathlib import Path
import numpy as np
from skimage import io
from mouselight_code.src.visualize import napari_viewer

In [ ]:
import numpy as np
brain_offsets = {
    "10-01": [69445.19581378, 12917.40798423, 30199.63896704],
    "8-01": [70093.27584462, 15071.5958194, 29306.73645404],
}
vol_offsets = {
    "10-01": {
        1: [3944.427317, 1689.489974, 2904.058044],
        2: [7562.41721, 2517.659516, 6720.099583],
        3: [6440.344565, 3724.653335, 3315.921558],
        4: [3693.850008, 4690.851133, 4759.545202],
        5: [2176.050385, 4472.356622, 5422.519379],
        6: [5186.880558, 1607.205131, 5627.930585],
        7: [4474.380558, 3801.205131, 5641.030585],
        8: [8625.680558, 3461.805131, 7853.730585],
        9: [8036.380558, 2739.005131, 7646.730585],
        10: [8908.480558, 2241.305131, 5275.430585],
        11: [5763.576767, 920.389294, 6949.146129],
        12: [4395.108079, 3142.101761, 7674.109968],
        13: [6357.017903, 3962.134266, 1793.497497],
        14: [1290.816602, 3784.927683, 5489.762402],
        15: [3261.686282, 4042.901892, 2753.811915],
        16: [6434.371327, 3146.622337, 5511.826519],
        17: [8759.453985, 3140.594903, 8062.858693],
        18: [3647.85608, 4787.29009, 5026.415948],
        19: [6278.469831, 1981.820562, 2234.779833],
        20: [2202.332629, 3856.654157, 2746.457209],
        21: [6119.880378, 3134.567468, 4973.882337],
        22: [7348.575311, 1596.96885, 3394.721975],
        23: [6633.877456, 2001.108354, 7079.429486],
        24: [6226.801328, 4177.61506, 3591.197682],
        25: [6270.405961, 4555.535222, 3526.056004],
    },
    "8-01": {
        1: [3808.881423, 2359.223225, 5006.26702],
        2: [6889.089085, 2566.530453, 8891.683733],
        3: [7128.395228, 1863.63414, 9648.801312],
        4: [5482.231871, 4208.245402, 11971.55106],
        5: [6671.293606, 3960.755275, 9643.859292],
        6: [5762.10132, 3843.673284, 5200.517052],
        7: [6337.40132, 4651.873284, 3297.317052],
        8: [5744.50132, 2051.973284, 5214.317052],
        9: [4518.90132, 1426.273284, 6342.517052],
        10: [4765.10132, 4331.973284, 9779.817052],
        11: [8681.046946, 3866.08193, 9473.853778],
        12: [3311.148546, 2486.773487, 9180.297745],
        13: [4844.082155, 1247.496358, 9977.939894],
        14: [5799.812932, 3939.750578, 8438.994633],
        15: [4774.172495, 2561.964214, 4977.603299],
        16: [4698.885169, 2982.058156, 7392.274638],
        17: [5052.616098, 4900.487158, 2202.164446],
        18: [4697.092614, 6292.276653, 1738.6029],
        19: [3954.675928, 2249.329085, 10126.20052],
        20: [3368.211559, 4350.103211, 6341.601026],
        21: [2275.350296, 2058.764732, 9266.288906],
        22: [6348.036119, 2952.529814, 10122.2469],
        23: [3842.043698, 4089.827617, 7974.444682],
        24: [73194, 15978.7, 35029.3],
        25: [5878.386609, 2172.311862, 5313.66071],
    },
}

scales = {
    "10-01": np.array([298.66187, 301.37174, 1050.67223]),
    "8-01": np.array([298.75923295, 304.41589844, 988.40414663]),
}


type_to_date = {"validation": "10-01", "test": "8-01"}

In [ ]:
im_dir = Path("C:/Users/shrey/Downloads/test")

gfp_files = list(im_dir.glob("**/*-gfp.tif"))

swc_base_path = im_dir / "Manual-GT"

In [ ]:
# for every image
for im_num, im_path in enumerate(gfp_files):


    print(f"Image {im_num+1}/{len(gfp_files)}")
    print(im_path)
    
    f = im_path.parts[-1][:-8].split("_")
    image = f[0]
    date = type_to_date[image]
    num = int(f[1])

    #read appropriate parameters
    scale = scales[date]
    brain_offset = brain_offsets[date]
    vol_offset = vol_offsets[date][num]
    im_offset = np.add(brain_offset, vol_offset)

    lower = int(np.floor((num - 1) / 5) * 5 + 1)
    upper = int(np.floor((num - 1) / 5) * 5 + 5)
    dir1 = date + "_" + image + "_" + str(lower) + "-" + str(upper)
    dir2 = date + "_" + image + "_" + str(num)
    swc_path = swc_base_path / dir1 / dir2

    swc_files = list(swc_path.glob("**/*.swc"))
    im = io.imread(im_path, plugin="tifffile")
    print(f"Image shape: {im.shape}")

    paths_total = []
    for swc_num, swc in enumerate(swc_files):
        if "cube" in swc.parts[-1]:
            # skip the bounding box swc
            continue

        df, swc_offset, _, _, _ = read_swc.read_swc(swc)

        #compute the offset of the swc relative to the image
        offset_diff = np.subtract(swc_offset, im_offset)
        G = df_to_graph(df)

        paths = graph_to_paths(G)

        # for every path in that swc
        
        for path_num, p in enumerate(paths):
            #convert from spatial coordinates to voxel coordinates
            pvox = (p + offset_diff) / (scale) * 1000
            paths_total.append(pvox)
            
        #napari_viewer(np.swapaxes(im,0,2), shapes=paths_total)

In [ ]:
print(paths[0][0])
print(paths_total[0][0])